#Framework for Datamining Twitter Using an Object Oriented Style Scales

## Authenticate

Replace the app_key, app_secret, oauth_token, and oauth_token_secret from your twitter application. 

In [ ]:
def createKeys():
    app_key='XX'      
    app_secret='XX'
    oauth_token='XX'
    oauth_token_secret='XX'
    return app_key,app_secret,oauth_token, oauth_token_secret

## Search

In [ ]:
#Base code by Dr. Gregory Saxton. Twitter: @gregorysaxton, website: social-metrics.org
#import sys
#sys.path.insert(0, './Imports') #adds Imports folder to path
#from Auth import * #import all variables stored in Imports/Keys.py 

def get_data_search(twitter_auth, search_keyword, tweets_toGather, max_id=None):
    try:
        '''
        NOTES FROM https://dev.twitter.com/docs/api/1.1/get/search/tweets 
        'count' = 'The number of tweets to return per page, up to a maximum of 100 per request. Defaults to 15.' 
         'result_type' options:
              * mixed: Include both popular and real time results in the response.
              * recent: return only the most recent results in the response
              * popular: return only the most popular results in the response.
        https://dev.twitter.com/docs/api/1.1/get/search/tweets
        '''
        twitter_dictResponse = twitter_auth.search(q=search_keyword, count = tweets_toGather, result_type = 'recent') 
        
    except Exception, e:
        print "Error reading id %s, exception: %s" % (search_keyword, e)
        return None
        
    print "GRABBED: ", len(twitter_dictResponse['statuses']), "statuses****"
    print "max_id VALUE USED FOR THIS GRAB-->", max_id
    print "returned twitter response as ", type(twitter_dictResponse)," for keyword: ", search_keyword
    return search_keyword, twitter_dictResponse

##Classes

In [ ]:
## Tweet class
class Tweet:
   'Common base class for all Tweets'

   def __init__(self, search_term, tweet_json):
      self.searchTerm = search_term
      self.tweetID = tweet_json[u'id']
      self.text = tweet_json[u'text'].encode('ascii', 'ignore')
      self.retweet_count = tweet_json[u'retweet_count']
      self.fav_count = tweet_json[u'favorite_count']
      self.entities_list = tweet_json[u'entities']
      self.url_list = tweet_json[u'entities'][u'urls']
      self. hashtag_list =  tweet_json[u'entities'][u'hashtags']
      self.mentions_list = tweet_json[u'entities'][u'user_mentions']
      self.coordinates = tweet_json[u'coordinates']
        
   def display_all(self):
      print "\nsearched: ", self.searchTerm,"\nFound:", self.text, '\n     RT: ', self.retweet_count, 'Fav: ', self.fav_count 

   
   def table_attributes(self):
      return dict(searchTerm= str(self.searchTerm), tweetID = self.tweetID, text = self.text, 
                 retweet_count =self.retweet_count,fav_count = self.fav_count)
 
    ## TODO - create url, entities, hashtags, mentions, and coordinate factories
        

##Helper Methods to Create Tweet_List with Tweet Class

In [ ]:
def TweetObj_from_StatusList(search_term, status_list):
    ## create tweet object for each status and save to a list of tweets
    print "creating tweet objects from status list"
    wait_dots = '.'
    print "working\n", wait_dots
    for entry in status_list:
        wait_dots = wait_dots + wait_dots
        print wait_dots
        tweet_list.append(Tweet(search_term, entry))
    return tweet_list
#call from TweetObj_from_StatusList(keyword_output['statuses']) 

##for debugging
#for tweet in tweet_list:
#    tweet.display_all() 


## Helper Method to Create SQL from Tweet

In [ ]:
#as transaction called tx, commits once all tweets inserted to tx
def toSQL(tweet_list):
    with dataset.connect('sqlite:///BasicTweet3.db') as tx:
        try:
            tx.begin()
            for tweet in tweet_list:
                tx['Tweet'].insert(tweet.table_attributes()) 
            tx.commit()
            print "Tweets in your sqlite database"
        except Exception, e:
            print "Error in uploading to sq, exception: %s" % (e)
            tx.rollback() #discard additions

## Main

In [ ]:
import dataset
from twython import Twython

tweet_list = []

def  main():
    app_key,app_secret,oauth_token, oauth_token_secret = createKeys()
    twitter_auth = Twython(app_key=app_key, app_secret=app_secret, 
                       oauth_token=oauth_token, oauth_token_secret=oauth_token_secret)
    print "authenticating"
    keyword_output = get_data_search(twitter_auth, 'behaviordots', 4)
    tweet_list = TweetObj_from_StatusList(keyword_output[0] , keyword_output[1]['statuses'])
    print tweet_list
    toSQL(tweet_list)

In [ ]:
if __name__ == "__main__":
    main()